title: "Individual Planning Report — Data Science Project"
author: Khush Shah
student_id: 39772439

---

Project: Predicting Usage of a Video Game Research Server  
**Question 1:** What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter?
